# Pandas and duckdb

## Task 0

In [ ]:
import duckdb
from pathlib import Path

duckdb_path = "data/sakila.duckdb"
Path(duckdb_path).unlink(missing_ok=True)

with duckdb.connect(duckdb_path) as conn, open("sql/load_sakila.sql") as ingest_script:
    conn.sql(ingest_script.read())

    description = conn.sql("DESC;").df()
    films = conn.sql("FROM film;").df()

films.head()

In [ ]:
dfs = {}

with duckdb.connect(duckdb_path) as conn:
    for name in description["name"]:
        dfs[name] = conn.sql(f"FROM {name};").df()

dfs.keys()

## Task 1

In [ ]:
long_movies = duckdb.sql("""
SELECT title, length AS duration
FROM films 
WHERE length > 180; 
""").df()

long_movies.head()

In [ ]:
love_word = duckdb.sql("""
SELECT title, rating, length, description 
FROM films
WHERE title ILIKE '%love%'; 
""").df()

love_word.head()

In [21]:
aggregate_function = duckdb.sql("""
SELECT 
    MIN(length) AS min_length_movies, 
    ROUND(AVG(length)) AS mean_length_movies, 
    MEDIAN(length) AS median_length_movies, 
    MAX(length) AS max_length_movies
FROM films
""").df()


aggregate_function

,min_length_movies,mean_length_movies,median_length_movies,max_length_movies
0,46,115.0,114.0,185


In [40]:
rental_rate = duckdb.sql("""
SELECT title, rental_rate, rental_duration, (rental_rate / rental_duration) AS cost_per_day                
FROM films
ORDER BY cost_per_day DESC;
""").df()

rental_rate.head(10)

,title,rental_rate,rental_duration,cost_per_day
0,PATHS CONTROL,4.99,3,1.663333
1,BACKLASH UNDEFEATED,4.99,3,1.663333
2,BEHAVIOR RUNAWAY,4.99,3,1.663333
3,VIRTUAL SPOILERS,4.99,3,1.663333
4,TEEN APOLLO,4.99,3,1.663333
5,TELEGRAPH VOYAGE,4.99,3,1.663333
6,BILKO ANONYMOUS,4.99,3,1.663333
7,HEARTBREAKERS BRIGHT,4.99,3,1.663333
8,CLERKS ANGELS,4.99,3,1.663333
9,TIES HUNGER,4.99,3,1.663333


In [42]:
description

,database,schema,name,column_names,column_types,temporary
0,sakila,main,actor,"[actor_id, first_name, last_name, last_update]","[DOUBLE, VARCHAR, VARCHAR, TIMESTAMP]",False
1,sakila,main,address,"[address_id, address, address2, district, city...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, VA...",False
2,sakila,main,category,"[category_id, name, last_update]","[BIGINT, VARCHAR, TIMESTAMP]",False
3,sakila,main,city,"[city_id, city, country_id, last_update]","[BIGINT, VARCHAR, BIGINT, TIMESTAMP]",False
4,sakila,main,country,"[country_id, country, last_update]","[BIGINT, VARCHAR, TIMESTAMP]",False
5,sakila,main,customer,"[customer_id, store_id, first_name, last_name,...","[BIGINT, BIGINT, VARCHAR, VARCHAR, VARCHAR, BI...",False
6,sakila,main,customer_list,"[ID, name, address, zip_code, phone, city, cou...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, VARCHAR, V...",False
7,sakila,main,film,"[film_id, title, description, release_year, la...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, BI...",False
8,sakila,main,film_actor,"[actor_id, film_id, last_update]","[BIGINT, BIGINT, TIMESTAMP]",False
9,sakila,main,film_category,"[film_id, category_id, last_update]","[BIGINT, BIGINT, TIMESTAMP]",False


In [ ]:
top_10_actors = duckdb.sql("""
SELECT a.actor_id, a.first_name, a.last_name, COUNT(*) AS film_count                
FROM films_joined 
JOIN film_actor fa ON a.actor_id = fa.actor_id
GROUP BY a.actor_id, a.first_name, a.last_name
ORDER BY fil_count DESC;
""").df()

top_10_actors

CatalogException: Catalog Error: Table with name actor does not exist!
Did you mean "pg_attrdef"?